In [2]:
!pip install -q lxml
!pip install webdriver-manager

import bs4
import lxml
import pandas
import urllib

from urllib import request



In [3]:
url_ligue_1 = "https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_2019-2020"
    
request_text = request.urlopen(url_ligue_1).read()
#print(request_text[:1000])
page = bs4.BeautifulSoup(request_text, "lxml")


In [4]:
print(page.find("title"))

<title>Championnat de France de football 2019-2020 — Wikipédia</title>


In [5]:
#trouver le tableau
# on identifie le tableau en question : c'est le premier qui a cette classe "wikitable sortable"
tableau_participants = page.find('table', {'class' : 'wikitable sortable'})
#print(tableau_participants)

In [6]:
#On recherche d’abord toutes les lignes du tableau avec la balise tr
table_body = tableau_participants.find('tbody')
rows = table_body.find_all('tr')

In [7]:
#On obtient une liste où chaque élément est une des lignes du tableau Pour illustrer cela, on va d’abord afficher la première ligne. 
#print(rows[0])

In [8]:
#La seconde ligne
#print(rows[1])

In [9]:
# EXEMPLE Nettoyer les sorties en ne gardant que le texte sur une ligne
cols = rows[1].find_all('td')
print(cols[0])
print(cols[0].text.strip())
for ele in cols : 
    print(ele.text.strip())

<td><a href="/wiki/Paris_Saint-Germain_Football_Club" title="Paris Saint-Germain Football Club">Paris Saint-Germain</a>
</td>
Paris Saint-Germain
Paris Saint-Germain
1974
637
1er
Thomas Tuchel
2018
Parc des Princes
47 929
46


In [10]:
# GENERALISATION
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    print(cols)

[]
['Paris Saint-Germain', '1974', '637', '1er', 'Thomas Tuchel', '2018', 'Parc des Princes', '47\xa0929', '46']
['LOSC Lille', '2000', '120', '2e', 'Christophe Galtier', '2017', 'Stade Pierre-Mauroy', '49\xa0712', '59']
['Olympique lyonnais', '1989', '310', '3e', 'Rudi Garcia', '2019', 'Groupama Stadium', '57\xa0206', '60']
['AS Saint-Étienne', '2004', '100', '4e', 'Claude Puel', '2019', 'Stade Geoffroy-Guichard', '41\xa0965', '66']
['Olympique de Marseille', '1996', '110', '5e', 'André Villas-Boas', '2019', 'Orange Vélodrome', '66\xa0226', '69']
['Montpellier HSC', '2009', '40', '6e', 'Michel Der Zakarian', '2017', 'Stade de la Mosson', '22\xa0000', '27']
['OGC Nice', '2002', '50', '7e', 'Patrick Vieira', '2018', 'Allianz Riviera', '35\xa0596', '60']
['Stade de Reims', '2018', '45', '8e', 'David Guion', '2017', 'Stade Auguste-Delaune', '20\xa0546', '35']
['Nîmes Olympique', '2018', '27', '9e', 'Bernard Blaquart', '2015', 'Stade des Costières', '15\xa0788', '35']
['Stade rennais FC', 

In [11]:
#On va mettre tout le contenu dans un dictionnaire, pour le transformer ensuite en DataFrame pandas :
dico_participants = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : 
        dico_participants[cols[0]] = cols[1:]
dico_participants

{'Paris Saint-Germain': ['1974',
  '637',
  '1er',
  'Thomas Tuchel',
  '2018',
  'Parc des Princes',
  '47\xa0929',
  '46'],
 'LOSC Lille': ['2000',
  '120',
  '2e',
  'Christophe Galtier',
  '2017',
  'Stade Pierre-Mauroy',
  '49\xa0712',
  '59'],
 'Olympique lyonnais': ['1989',
  '310',
  '3e',
  'Rudi Garcia',
  '2019',
  'Groupama Stadium',
  '57\xa0206',
  '60'],
 'AS Saint-Étienne': ['2004',
  '100',
  '4e',
  'Claude Puel',
  '2019',
  'Stade Geoffroy-Guichard',
  '41\xa0965',
  '66'],
 'Olympique de Marseille': ['1996',
  '110',
  '5e',
  'André Villas-Boas',
  '2019',
  'Orange Vélodrome',
  '66\xa0226',
  '69'],
 'Montpellier HSC': ['2009',
  '40',
  '6e',
  'Michel Der Zakarian',
  '2017',
  'Stade de la Mosson',
  '22\xa0000',
  '27'],
 'OGC Nice': ['2002',
  '50',
  '7e',
  'Patrick Vieira',
  '2018',
  'Allianz Riviera',
  '35\xa0596',
  '60'],
 'Stade de Reims': ['2018',
  '45',
  '8e',
  'David Guion',
  '2017',
  'Stade Auguste-Delaune',
  '20\xa0546',
  '35'],
 'Nîme

In [12]:
# Formation nouveau tableau
data_participants = pandas.DataFrame.from_dict(dico_participants,orient='index')
data_participants.head()

,0,1,2,3,4,5,6,7
Paris Saint-Germain,1974,637,1er,Thomas Tuchel,2018,Parc des Princes,47 929,46
LOSC Lille,2000,120,2e,Christophe Galtier,2017,Stade Pierre-Mauroy,49 712,59
Olympique lyonnais,1989,310,3e,Rudi Garcia,2019,Groupama Stadium,57 206,60
AS Saint-Étienne,2004,100,4e,Claude Puel,2019,Stade Geoffroy-Guichard,41 965,66
Olympique de Marseille,1996,110,5e,André Villas-Boas,2019,Orange Vélodrome,66 226,69


In [13]:
# récupération entête
for row in rows:
    cols = row.find_all('th')
    print(cols)
    if len(cols) > 0 : 
        cols = [ele.get_text(separator=' ').strip().title() for ele in cols]
        columns_participants = cols

[<th scope="col">Club
</th>, <th scope="col">Dernière<br/>montée
</th>, <th scope="col">Budget<sup class="reference" id="cite_ref-3"><a href="#cite_note-3"><span class="cite_crochet">[</span>3<span class="cite_crochet">]</span></a></sup><br/>en M<a href="/wiki/Euro" title="Euro">€</a>
</th>, <th scope="col">Classement<br/><a href="/wiki/Championnat_de_France_de_football_2018-2019" title="Championnat de France de football 2018-2019">2018-2019</a>
</th>, <th scope="col">Entraîneur
</th>, <th scope="col">Depuis
</th>, <th scope="col">Stade
</th>, <th scope="col">Capacité<br/>en L1<sup class="reference" id="cite_ref-4"><a href="#cite_note-4"><span class="cite_crochet">[</span>4<span class="cite_crochet">]</span></a></sup>
</th>, <th scope="col">Nombre<br/>de saisons<br/>en L1
</th>]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [14]:
columns_participants

['Club',
 'Dernière Montée',
 'Budget [ 3 ] En M €',
 'Classement 2018-2019',
 'Entraîneur',
 'Depuis',
 'Stade',
 'Capacité En L1 [ 4 ]',
 'Nombre De Saisons En L1']

In [15]:
#finalisation du tableau
data_participants.columns = columns_participants[1:]
data_participants.head()

,Dernière Montée,Budget [ 3 ] En M €,Classement 2018-2019,Entraîneur,Depuis,Stade,Capacité En L1 [ 4 ],Nombre De Saisons En L1
Paris Saint-Germain,1974,637,1er,Thomas Tuchel,2018,Parc des Princes,47 929,46
LOSC Lille,2000,120,2e,Christophe Galtier,2017,Stade Pierre-Mauroy,49 712,59
Olympique lyonnais,1989,310,3e,Rudi Garcia,2019,Groupama Stadium,57 206,60
AS Saint-Étienne,2004,100,4e,Claude Puel,2019,Stade Geoffroy-Guichard,41 965,66
Olympique de Marseille,1996,110,5e,André Villas-Boas,2019,Orange Vélodrome,66 226,69
